In [1]:
with open('libraries.py') as f:
    code = f.read()
exec(code)

with open('functions.py') as f:
    code = f.read()
exec(code)

In [2]:
# determine user
user = getpass.getuser()
if user == 'peymansh':
    main_folder_path = '/Users/peymansh/Dropbox (MIT)/Research/AI and Occupations/ai-exposure'
    data_path = f'{main_folder_path}/output'

In [3]:
# Pick occupation and initialize variables
occupation = 'travelAgents'
occupation = 'insuranceUnderwriters'
# occupation = 'pileDriverOperators'
# occupation = 'dredgeOperators'
# occupation = 'gradersAndSorters'
# occupation = 'reinforcingIron'
# occupation = 'insuranceAppraisers'
# occupation = 'floorSanders'
# occupation = 'dataEntryKeyer'
# # occupation = 'athletesAndSportsCompetitors'
# # # occupation = 'shampooers'

GPT_input_occupation, plot_title_occupation, occupation_code, occupation_folder = pick_occupation(occupation)

In [4]:
# Manual DAG only available for travel agents and insurance underwriters
# if working w/ these two occupations include manual DAG too, else discard relevant code
manual_present = 0
if occupation == 'travelAgents' or occupation == 'insuranceUnderwriters':
    manual_present = 1

### Read Data

In [5]:
suffix = ''

if manual_present == 1:
    manual_df = pd.read_csv(f'{occupation_folder}/{occupation}_costMin_{suffix}manual.csv')
firstLastTask_df = pd.read_csv(f'{occupation_folder}/{occupation}_costMin_firstLastTask.csv')
firstLastTask_conditioned_df = pd.read_csv(f'{occupation_folder}/{occupation}_costMin_{suffix}firstLastTask_conditioned.csv')
partitioned_df = pd.read_csv(f'{occupation_folder}/{occupation}_costMin_partitioned.csv')
partitioned_conditioned_df = pd.read_csv(f'{occupation_folder}/{occupation}_costMin_{suffix}partitioned_conditioned.csv')


In [6]:
# get number of tasks in occupation
onet = pd.read_csv(f'{data_path}/data/onet_occupations_yearly.csv')
onet = onet.sort_values(by=['year', 'occ_code', 'occ_title', 'task_id'])
onet = onet[onet['year'] == 2023].reset_index(drop=True)

# Get list of tasks
my_df = onet[(onet.occ_code == f'{occupation_code}') & (onet.year == 2023)]
tasks = my_df['task'].unique().tolist()
num_tasks = len(tasks)

In [7]:
def get_manual_to_machine_tasks(df):
    # Convert the string representation of lists into actual Python lists
    for col in ['optimal_scheme', 'optimal_scheme_manual_tasks']:
        df[col] = df[col].apply(ast.literal_eval)

    # Define a function to subtract lists in the next row from lists in the current row
    def subtract_lists(current_list, next_list):
        # Flatten lists for easier comparison
        current_flat = [item for sublist in current_list for item in sublist]
        next_flat = [item for sublist in next_list for item in sublist]
        
        # Subtract elements
        result_flat = [item for item in current_flat if item not in next_flat]
        
        # Rebuild the list of lists structure
        result_list = [[item] for item in result_flat]
        return result_list

    # Create the new column by applying the function to each row
    new_column = [[]]
    for i in range(1, len(df)):
        current_list = df.loc[i - 1, 'optimal_scheme_manual_tasks']
        next_list = df.loc[i, 'optimal_scheme_manual_tasks']
        new_column.append(subtract_lists(current_list, next_list))

    # Assign the new column to the DataFrame
    df['newly_automated_tasks'] = new_column
    
    # Replace empty lists with an empty string
    df['newly_automated_tasks'] = df['newly_automated_tasks'].apply(lambda x: '' if x == [] else x)

    return df

firstLastTask_df = get_manual_to_machine_tasks(firstLastTask_df)
firstLastTask_conditioned_df = get_manual_to_machine_tasks(firstLastTask_conditioned_df)
partitioned_df = get_manual_to_machine_tasks(partitioned_df)
partitioned_conditioned_df = get_manual_to_machine_tasks(partitioned_conditioned_df)

In [8]:
# Function to calculate the average length of automated chain
def calc_avg_automationPlan_size(row):
    all_partitions = row['optimal_scheme']
    manual_tasks = row['optimal_scheme_manual_tasks']
    
    # Calculate the lengths of each sublist
    if len(all_partitions) != len(manual_tasks):
        length = [len(partition) for partition in all_partitions if partition not in manual_tasks]
    else:
        return 0
    
    # Compute the average length
    if length:  # Check to avoid division by zero
        return sum(length) / len(length)
    else:
        return 0
    

# Function to calculate the max length of automated chain
def calc_max_automationPlan_size(row):
    all_partitions = row['optimal_scheme']
    manual_tasks = row['optimal_scheme_manual_tasks']
    
    # Calculate the lengths of each sublist
    if len(all_partitions) != len(manual_tasks):
        length = [len(partition) for partition in all_partitions if partition not in manual_tasks]
    else:
        return 0
    
    # Compute the average length
    if length:  # Check to avoid division by zero
        return max(length)
    else:
        return 0
    

def substitute_lists(row):
    optimal_scheme_str = 'optimal_scheme'
    optimal_scheme_manual_tasks_str = 'optimal_scheme_manual_tasks'
    
    # Convert string representations of lists into actual lists
    list_a = row[optimal_scheme_str]
    list_b = row[optimal_scheme_manual_tasks_str]
    # list_a = ast.literal_eval(row[optimal_scheme_str])
    # list_b = ast.literal_eval(row[optimal_scheme_manual_tasks_str])
    
    # Iterate over elements in list_b
    for sublist_b in list_b:
        # Replace matching lists in list_a with the formatted string
        list_a = [
            f"<{sublist_b[0]}>" if sublist == sublist_b else sublist
            for sublist in list_a
        ]
    
    # Convert the list back to a string without quotes around "<x>"
    return "[" + ", ".join(
        str(item) if isinstance(item, list) else item
        for item in list_a
    ) + "]"


In [9]:
firstLastTask_df['avg_automationChain_size'] = firstLastTask_df.apply(calc_avg_automationPlan_size, axis=1)
firstLastTask_conditioned_df['avg_automationChain_size'] = firstLastTask_conditioned_df.apply(calc_avg_automationPlan_size, axis=1)
partitioned_df['avg_automationChain_size'] = partitioned_df.apply(calc_avg_automationPlan_size, axis=1)
partitioned_conditioned_df['avg_automationChain_size'] = partitioned_conditioned_df.apply(calc_avg_automationPlan_size, axis=1)

firstLastTask_df['max_automationChain_size'] = firstLastTask_df.apply(calc_max_automationPlan_size, axis=1)
firstLastTask_conditioned_df['max_automationChain_size'] = firstLastTask_conditioned_df.apply(calc_max_automationPlan_size, axis=1)
partitioned_df['max_automationChain_size'] = partitioned_df.apply(calc_max_automationPlan_size, axis=1)
partitioned_conditioned_df['max_automationChain_size'] = partitioned_conditioned_df.apply(calc_max_automationPlan_size, axis=1)


firstLastTask_df['pretty_name'] = firstLastTask_df.apply(substitute_lists, axis=1)
firstLastTask_conditioned_df['pretty_name'] = firstLastTask_conditioned_df.apply(substitute_lists, axis=1)
partitioned_df['pretty_name'] = partitioned_df.apply(substitute_lists, axis=1)
partitioned_conditioned_df['pretty_name'] = partitioned_conditioned_df.apply(substitute_lists, axis=1)

### Create new dataset comparing DAGs stats

In [10]:
cols_to_keep = ['alpha', 'cost', 'pretty_name', 'newly_automated_tasks', 'optimal_schemes_count', 'avg_automationChain_size', 'max_automationChain_size']
if manual_present == 1:
    # prepare manual dataset
    manual_df = get_manual_to_machine_tasks(manual_df) 
    manual_df['avg_automationChain_size'] = manual_df.apply(calc_avg_automationPlan_size, axis=1)
    manual_df['max_automationChain_size'] = manual_df.apply(calc_max_automationPlan_size, axis=1)
    manual_df['pretty_name'] = manual_df.apply(substitute_lists, axis=1)
    
    # merge datasets
    comparison_df = pd.merge(manual_df[cols_to_keep], 
                            firstLastTask_df[cols_to_keep], 
                            on=['alpha'], 
                            how = 'outer',
                            suffixes=('_M', '_FLT'))
    comparison_df = pd.merge(comparison_df,
                            firstLastTask_conditioned_df[cols_to_keep], 
                            on=['alpha'], 
                            how = 'outer')
    comparison_df = pd.merge(comparison_df,
                            partitioned_df[cols_to_keep], 
                            on=['alpha'], 
                            how = 'outer',
                            suffixes=('_CFLT', '_P'))
    comparison_df = pd.merge(comparison_df,
                            partitioned_conditioned_df[cols_to_keep], 
                            on=['alpha'], 
                            how = 'outer')

    # change column names
    comparison_df = comparison_df.rename(columns={'pretty_name': 'optimal_scheme_CP',
                                                'cost': 'cost_CP', 
                                                'newly_automated_tasks': 'newly_automated_tasks_CP',
                                                'optimal_schemes_count': 'optimal_schemes_count_CP',
                                                'avg_automationChain_size': 'avg_automationChain_size_CP',
                                                'max_automationChain_size': 'max_automationChain_size_CP'})
else:
    # merge datasets
    comparison_df = pd.merge(firstLastTask_df[cols_to_keep], 
                            firstLastTask_conditioned_df[cols_to_keep], 
                            on=['alpha'], 
                            how = 'outer',
                            suffixes=('_FLT', '_CFLT'))
    comparison_df = pd.merge(comparison_df,
                            partitioned_df[cols_to_keep], 
                            on=['alpha'], 
                            how = 'outer')
    comparison_df = pd.merge(comparison_df,
                            partitioned_conditioned_df[cols_to_keep], 
                            on=['alpha'], 
                            how = 'outer')

    # change column names
    comparison_df = comparison_df.rename(columns={'pretty_name_x': 'optimal_scheme_P',
                                                'pretty_name_y': 'optimal_scheme_CP', 
                                                'cost_x': 'cost_P',
                                                'cost_y': 'cost_CP', 
                                                'newly_automated_tasks_x': 'newly_automated_tasks_P',
                                                'newly_automated_tasks_y': 'newly_automated_tasks_CP',
                                                'optimal_schemes_count_x': 'optimal_schemes_count_P',
                                                'optimal_schemes_count_y': 'optimal_schemes_count_CP',
                                                'avg_automationChain_size_x': 'avg_automationChain_size_P',
                                                'avg_automationChain_size_y': 'avg_automationChain_size_CP',
                                                'max_automationChain_size_x': 'max_automationChain_size_P',
                                                'max_automationChain_size_y': 'max_automationChain_size_CP'})

# hacky way to fix column names after merge
comparison_df.columns = comparison_df.columns.str.replace('pretty_name', 'optimal_scheme')

for col in comparison_df.columns:
    if "optimal_scheme_" in col:
        # Create a new column indicating where the value changes
        change_col = col + '_change'
        comparison_df[change_col] = (comparison_df[col] != comparison_df[col].shift()).cumsum() - 1


# # change ordering of columns
# sorted_cols = sorted(comparison_df.columns, key=lambda x: x.split('_')[-1])
# comparison_df = comparison_df[sorted_cols]

# save output
comparison_df.to_csv(f'{occupation_folder}/{occupation}_costMin_DAGcomparison_df.csv', index=False)

In [11]:
comparison_df

,alpha,cost_M,optimal_scheme_M,newly_automated_tasks_M,optimal_schemes_count_M,avg_automationChain_size_M,max_automationChain_size_M,cost_FLT,optimal_scheme_FLT,newly_automated_tasks_FLT,...,optimal_scheme_CP,newly_automated_tasks_CP,optimal_schemes_count_CP,avg_automationChain_size_CP,max_automationChain_size_CP,optimal_scheme_M_change,optimal_scheme_FLT_change,optimal_scheme_CFLT_change,optimal_scheme_P_change,optimal_scheme_CP_change
0,0.00,420.00,"[<0>, <1>, <2>, <3>, <4>, <5>, <6>]",,1,0.00,0,420.00,"[<0>, <1>, <2>, <3>, <4>, <5>, <6>]",,...,"[<0>, <1>, <2>, <3>, <4>, <5>, <6>]",,1,0.00,0,0,0,0,0,0
1,0.00,420.00,"[<0>, <1>, <2>, <3>, <4>, <5>, <6>]",,1,0.00,0,420.00,"[<0>, <1>, <2>, <3>, <4>, <5>, <6>]",,...,"[<0>, <1>, <2>, <3>, <4>, <5>, <6>]",,1,0.00,0,0,0,0,0,0
2,0.00,420.00,"[<0>, <1>, <2>, <3>, <4>, <5>, <6>]",,1,0.00,0,420.00,"[<0>, <1>, <2>, <3>, <4>, <5>, <6>]",,...,"[<0>, <1>, <2>, <3>, <4>, <5>, <6>]",,1,0.00,0,0,0,0,0,0
3,0.00,420.00,"[<0>, <1>, <2>, <3>, <4>, <5>, <6>]",,1,0.00,0,420.00,"[<0>, <1>, <2>, <3>, <4>, <5>, <6>]",,...,"[<0>, <1>, <2>, <3>, <4>, <5>, <6>]",,1,0.00,0,0,0,0,0,0
4,0.00,420.00,"[<0>, <1>, <2>, <3>, <4>, <5>, <6>]",,1,0.00,0,420.00,"[<0>, <1>, <2>, <3>, <4>, <5>, <6>]",,...,"[<0>, <1>, <2>, <3>, <4>, <5>, <6>]",,1,0.00,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.00,20.26,"[[0], [1, 2, 3, 4, 5, 6]]",,1,3.50,6,45.24,"[[0, 2, 3, 4, 6], [1], [5]]",,...,"[[0, 1, 3, 4, 5, 6], <2>]",,1,6.00,6,58,85,7,183,9
996,1.00,20.19,"[[0], [1, 2, 3, 4, 5, 6]]",,1,3.50,6,45.18,"[[0, 2, 3, 4, 6], [1], [5]]",,...,"[[0, 1, 3, 4, 5, 6], <2>]",,1,6.00,6,58,85,7,183,9
997,1.00,20.13,"[[0], [1, 2, 3, 4, 5, 6]]",,1,3.50,6,45.12,"[[0, 2, 3, 4, 6], [1], [5]]",,...,"[[0, 1, 3, 4, 5, 6], <2>]",,1,6.00,6,58,85,7,183,9
998,1.00,20.06,"[[0], [1, 2, 3, 4, 5, 6]]",,1,3.50,6,45.06,"[[0, 2, 3, 4, 6], [1], [5]]",,...,"[[0, 1, 3, 4, 5, 6], <2>]",,1,6.00,6,58,85,7,183,9


## Plots

In [12]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10, 12))
fig.suptitle(f'Comparison of Optimal Execution Plans for Different DAGs\n\nOccupation: {plot_title_occupation} ({num_tasks} Tasks)\n', fontsize=16)

# Plot data on the first subplot
ax1.plot(comparison_df.alpha, comparison_df.cost_FLT, label = 'First-Last Task')
ax1.plot(comparison_df.alpha, comparison_df.cost_CFLT, label = '(Conditioned) First-Last Task')
ax1.plot(comparison_df.alpha, comparison_df.cost_P, label = 'Partitioned')
ax1.plot(comparison_df.alpha, comparison_df.cost_CP, label = '(Conditioned) Partitioned')
if manual_present == 1:
    ax1.plot(comparison_df.alpha, comparison_df.cost_M, label = 'Manual')

ax1.set_title('Optimal Execution Plan Cost')
#ax1.set_xlabel(r'AI Quality ($\alpha$)')
ax1.set_ylabel('Cost (in minutes)')
ax1.legend(loc='upper right')


# Plot data on the second subplot
ax2.plot(comparison_df.alpha, comparison_df.optimal_scheme_FLT_change, label = 'First-Last Task')
ax2.plot(comparison_df.alpha, comparison_df.optimal_scheme_CFLT_change, label = '(Conditioned) First-Last Task')
ax2.plot(comparison_df.alpha, comparison_df.optimal_scheme_P_change, label = 'Partitioned')
ax2.plot(comparison_df.alpha, comparison_df.optimal_scheme_CP_change, label = '(Conditioned) Partitioned')
if manual_present == 1:
    ax2.plot(comparison_df.alpha, comparison_df.optimal_scheme_M_change, label = 'Manual')

ax2.set_title('Optimal Execution Plan Switches: Incremental Count')
#ax2.set_xlabel(r'AI Quality ($\alpha$)')
ax2.set_ylabel('Incremental Count of Switches')
ax2.legend(loc='upper left')


# Plot data on the third subplot
#ax3.axhline(y=-0, color='red', linestyle='--', alpha=0.9)
ax3.plot(firstLastTask_df.alpha, firstLastTask_df.manual_tasks_count, label = 'First-Last Task')
ax3.plot(firstLastTask_conditioned_df.alpha, firstLastTask_conditioned_df.manual_tasks_count, label = '(Conditioned) First-Last Task')
ax3.plot(partitioned_df.alpha, partitioned_df.manual_tasks_count, label = 'Partitioned')
ax3.plot(partitioned_conditioned_df.alpha, partitioned_conditioned_df.manual_tasks_count, label = '(Conditioned) Partitioned')
if manual_present == 1:
    ax3.plot(manual_df.alpha, manual_df.manual_tasks_count, label = 'Manual')

ax3.set_title('Count of Non-automated Tasks in Optimal Execution Plan(s)')
#ax3.set_xlabel(r'AI Quality ($\alpha$)')
ax3.set_ylabel('Count of Non-automated Tasks')
ax3.legend(loc='upper right')


# Plot data on the fourth subplot
ax4.plot(comparison_df.alpha, comparison_df.optimal_schemes_count_FLT, label = 'First-Last Task')
ax4.plot(comparison_df.alpha, comparison_df.optimal_schemes_count_CFLT, label = '(Conditioned) First-Last Task')
ax4.plot(comparison_df.alpha, comparison_df.optimal_schemes_count_P, label = 'Partitioned')
ax4.plot(comparison_df.alpha, comparison_df.optimal_schemes_count_CP, label = '(Conditioned) Partitioned')
if manual_present == 1:
    ax4.plot(comparison_df.alpha, comparison_df.optimal_schemes_count_M, label = 'Manual')

ax4.set_title('Number of Optimal Execution Plans')
ax4.set_xlabel(r'AI Quality ($\alpha$)')
ax4.set_ylabel('Number of Optimal Schemes')
ax4.set_yticks([0, 1, 2, 3])
ax4.legend(loc='upper left')

# Adjust the layout
plt.tight_layout()

# Save the plot
plt.savefig(f'{occupation_folder}/{occupation}_costMin_graph.png', dpi=300)
plt.savefig(f'{data_path}/daily_tasks_occupations_analysis/plots/{occupation}_costMin_graph.png', dpi=300)

# Close figure
plt.close()

In [13]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(10, 12))
fig.suptitle(f'Comparison of Optimal Execution Plans for Different DAGs\n\nOccupation: {plot_title_occupation} ({num_tasks} Tasks)\n', fontsize=16)

# Plot data on the first subplot
ax1.plot(comparison_df.alpha, comparison_df.avg_automationChain_size_FLT, label = 'First-Last Task')
ax1.plot(comparison_df.alpha, comparison_df.avg_automationChain_size_CFLT, label = '(Conditioned) First-Last Task')
ax1.plot(comparison_df.alpha, comparison_df.avg_automationChain_size_P, label = 'Partitioned')
ax1.plot(comparison_df.alpha, comparison_df.avg_automationChain_size_CP, label = '(Conditioned) Partitioned')
if manual_present == 1:
    ax1.plot(comparison_df.alpha, comparison_df.avg_automationChain_size_M, label = 'Manual')

ax1.set_title('Avg. Size of Automated Chains in Optimal Execution Plan')
#ax1.set_xlabel(r'AI Quality ($\alpha$)')
ax1.set_ylabel('Avg. Size of Automated Chains')
ax1.legend(loc='upper left')


# Plot data on the second subplot
ax2.plot(comparison_df.alpha, comparison_df.optimal_scheme_FLT_change, label = 'First-Last Task')
ax2.plot(comparison_df.alpha, comparison_df.optimal_scheme_CFLT_change, label = '(Conditioned) First-Last Task')
ax2.plot(comparison_df.alpha, comparison_df.optimal_scheme_P_change, label = 'Partitioned')
ax2.plot(comparison_df.alpha, comparison_df.optimal_scheme_CP_change, label = '(Conditioned) Partitioned')
if manual_present == 1:
    ax2.plot(comparison_df.alpha, comparison_df.optimal_scheme_M_change, label = 'Manual')

ax2.set_title('Optimal Execution Plan Switches: Incremental Count')
#ax2.set_xlabel(r'AI Quality ($\alpha$)')
ax2.set_ylabel('Incremental Count of Switches')
ax2.legend(loc='upper left')


# Plot data on the third subplot
#ax3.axhline(y=-0, color='red', linestyle='--', alpha=0.9)
ax3.plot(firstLastTask_df.alpha, firstLastTask_df.manual_tasks_count, label = 'First-Last Task')
ax3.plot(firstLastTask_conditioned_df.alpha, firstLastTask_conditioned_df.manual_tasks_count, label = '(Conditioned) First-Last Task')
ax3.plot(partitioned_df.alpha, partitioned_df.manual_tasks_count, label = 'Partitioned')
ax3.plot(partitioned_conditioned_df.alpha, partitioned_conditioned_df.manual_tasks_count, label = '(Conditioned) Partitioned')
if manual_present == 1:
    ax3.plot(manual_df.alpha, manual_df.manual_tasks_count, label = 'Manual')

ax3.set_title('Count of Non-automated Tasks in Optimal Execution Plan(s)')
#ax3.set_xlabel(r'AI Quality ($\alpha$)')
ax3.set_ylabel('Count of Non-automated Tasks')
ax3.legend(loc='upper right')


# Plot data on the fourth subplot
ax4.plot(comparison_df.alpha, comparison_df.max_automationChain_size_FLT, label = 'First-Last Task')
ax4.plot(comparison_df.alpha, comparison_df.max_automationChain_size_CFLT, label = '(Conditioned) First-Last Task')
ax4.plot(comparison_df.alpha, comparison_df.max_automationChain_size_P, label = 'Partitioned')
ax4.plot(comparison_df.alpha, comparison_df.max_automationChain_size_CP, label = '(Conditioned) Partitioned')
if manual_present == 1:
    ax4.plot(comparison_df.alpha, comparison_df.max_automationChain_size_M, label = 'Manual')

ax4.set_title('Max. Size of Automated Chains in Optimal Execution Plan')
ax4.set_xlabel(r'AI Quality ($\alpha$)')
ax4.set_ylabel('Max. Size of Automated Chains')
ax4.legend(loc='upper left')

# Adjust the layout
plt.tight_layout()

# Save the plot
plt.savefig(f'{occupation_folder}/{occupation}_costMin_graph.png', dpi=300)
plt.savefig(f'{data_path}/daily_tasks_occupations_analysis/plots/{occupation}_costMin_graph.png', dpi=300)

# Close figure
plt.close()